In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import cv2
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from PIL import Image
from lang_sam import LangSAM

/home/shuozhao/anaconda3/envs/lsa/lib/python3.9/site-packages/groundingdino/models/GroundingDINO/ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


In [3]:
# os.environ['http_proxy'] = "http://10.200.13.85:3128"
# os.environ['https_proxy'] = "http://10.200.13.85:3128"

In [4]:
def cutOutObj(img, mask, space=10):
    x, y, w, h = cv2.boundingRect(mask)
    cutout = img[y:y+h, x:x+w]
    cutout_brga = cv2.cvtColor(cutout, cv2.COLOR_RGB2BGRA)
    cutout_brga[...,3] = mask[y:y+h, x:x+w]
    n_rows, n_cols = cutout_brga.shape[:2]
    cutout_pad = np.zeros([n_rows+space*2, n_cols+space*2, 4], dtype=np.uint8)
    cutout_pad[space:n_rows+space, space:n_cols+space,:] = cutout_brga
    return cutout_pad

In [5]:
def contrastEnhance(img):
    b, g, r = cv2.split(img)
    b1 = cv2.equalizeHist(b)
    g1 = cv2.equalizeHist(g)
    r1 = cv2.equalizeHist(r)
    o_img = cv2.merge([b1,g1,r1])
    return o_img

In [6]:
model = LangSAM("vit_h", "/data2/shuozhao/DigitalAuto/sam_models/sam_vit_h_4b8939.pth")

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /ShilongLiu/GroundingDINO/resolve/main/GroundingDINO_SwinB.cfg.py (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7ffac91a5b20>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: f719d7e8-ba3f-4713-b81d-e41a67805503)')' thrown while requesting HEAD https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/GroundingDINO_SwinB.cfg.py
/home/shuozhao/anaconda3/envs/lsa/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7ffabf0db850>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: d117e7bd-9b42-491e-b27c-a1860894c264)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7ff981435dc0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: f35a5911-fc2c-49cf-a83a-b5d7fe64cd8e)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/config.json
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port

Model loaded from /home/shuozhao/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [17]:
slice_width = 2048
slice_height = 2048
slice_stride = 512
whole_img_path = Path("/data2/shuozhao/DigitalAuto/wutai/whole_img.jpg")
output_path = Path("pictures/cutouts/temple")
text_prompt = "temples"

In [12]:
whole_img = cv2.imread(str(whole_img_path))
whole_img = cv2.cvtColor(whole_img, cv2.COLOR_BGR2RGB)

In [13]:
img_height, img_width = whole_img.shape[:2]

In [18]:
for row in tqdm(range(0, img_height, slice_height-slice_stride)):
    for col in range(0, img_width, slice_width-slice_stride):
        slice_img = whole_img[row:row+slice_height, col:col+slice_width]
        img_aug = contrastEnhance(slice_img)
        pil_img = Image.fromarray(img_aug)
        
        masks, boxes, phrases, logits = model.predict(pil_img, text_prompt)
        masks = masks.numpy().astype(np.uint8)*255
        for idx, mask in enumerate(masks):
            img_bgra = cutOutObj(slice_img, mask)
            cv2.imwrite(str(output_path/f'row_{row}_col_{col}_mask_{idx}.png'),img_bgra)


  0%|          | 0/11 [00:00<?, ?it/s]